In [1]:
import time
import json
import math
import numpy as np
import nest_asyncio
import asyncio
import websockets
nest_asyncio.apply()

In [2]:
async def websocket_test(payload, indent=None):
    uri = "ws://127.0.0.1:8765/ws"
    async with websockets.connect(uri) as ws:
        await ws.send(json.dumps(payload))
        print("✅ send:", payload)

        try:
            # 5 秒以内に返事が来なければ TimeoutError
            response = await asyncio.wait_for(ws.recv(), timeout=5)
            if indent:
                print("📨 recv: ", json.dumps(json.loads(response), indent=indent, ensure_ascii=False))
                return json.loads(response)
            else:
                print("📨 recv:", response)
                return json.loads(response)
        except asyncio.TimeoutError:
            print("⏰ タイムアウト（5 秒以内にメッセージが来ませんでした）")

## camera object

In [3]:
payload = {
    "type": "add",
    "command": "camera",
    "name": "camera test",
}

await websocket_test(payload)

✅ send: {'type': 'add', 'command': 'camera', 'name': 'camera test'}
📨 recv: {"objectId":0,"message":"Camera added successfully with name: camera test","type":"success"}


{'objectId': 0,
 'message': 'Camera added successfully with name: camera test',
 'type': 'success'}

In [4]:
payload = {
  "type": "camera",
  "command": "switch",
  "cameraId": 0
}
await websocket_test(payload, indent=2)

✅ send: {'type': 'camera', 'command': 'switch', 'cameraId': 0}
📨 recv:  {
  "message": "Switched to camera 0",
  "type": "success"
}


{'message': 'Switched to camera 0', 'type': 'success'}

In [39]:
payload = {
  "type": "camera",
  "command": "free",
}
await websocket_test(payload, indent=2)

✅ send: {'type': 'camera', 'command': 'free'}
📨 recv:  {
  "type": "success",
  "message": "Switched to free camera mode"
}


{'type': 'success', 'message': 'Switched to free camera mode'}

## camera view

In [5]:
payload = {
    "type": "camera",
    "command": "getview"
}
await websocket_test(payload, indent=2)

✅ send: {'type': 'camera', 'command': 'getview'}
📨 recv:  {
  "pos": [
    -0.07363355,
    0.711534441,
    1.96424425
  ],
  "rot": [
    48.6000137,
    -105.899544,
    0.0
  ],
  "fov": 60.0,
  "mode": "object",
  "activeCameraId": 0,
  "message": "Current camera view retrieved",
  "type": "success"
}


{'pos': [-0.07363355, 0.711534441, 1.96424425],
 'rot': [48.6000137, -105.899544, 0.0],
 'fov': 60.0,
 'mode': 'object',
 'activeCameraId': 0,
 'message': 'Current camera view retrieved',
 'type': 'success'}

In [27]:


def calculate_circular_camera_view(
    frame: int,
    total_frames: int,
    radius: float,
    height: float,
    fov: float = 35.0
) -> dict:
    angle_rad = (frame / total_frames) * 2 * math.pi

    pos_x = radius * math.cos(angle_rad)
    pos_y = height
    pos_z = radius * math.sin(angle_rad)
    pos = [pos_x, pos_y, pos_z]

    pitch_rad = math.atan2(height, radius)
    pitch_deg = math.degrees(pitch_rad)

    yaw_rad = math.atan2(-pos_x, -pos_z)
    yaw_deg = math.degrees(yaw_rad)

    roll_deg = 0.0

    rot = [pitch_deg, yaw_deg, roll_deg]

    camera_data = {
        "type": "camera",
        "command": "setview",
        "pos": [round(p, 4) for p in pos],
        "rot": [round(r, 4) for r in rot],
        "fov": fov
    }

    return camera_data

CAMERA_RADIUS = 4.0
CAMERA_HEIGHT = 2.0
TOTAL_FRAMES_FOR_ONE_LOOP = 240
CAMERA_FOV = 30.0

frames = []
for i in range(240):
    frame = calculate_circular_camera_view(
        frame=i,
        total_frames=TOTAL_FRAMES_FOR_ONE_LOOP,
        radius=CAMERA_RADIUS,
        height=CAMERA_HEIGHT,
        fov=CAMERA_FOV,
    )
    frames.append(frame)

for f in frames:
    await websocket_test(f, indent=2)

✅ send: {'type': 'camera', 'command': 'setview', 'pos': [4.0, 2.0, 0.0], 'rot': [26.5651, -90.0, 0.0], 'fov': 30.0}
📨 recv:  {
  "type": "success",
  "message": "Camera view updated successfully"
}
✅ send: {'type': 'camera', 'command': 'setview', 'pos': [3.9986, 2.0, 0.1047], 'rot': [26.5651, -91.5, 0.0], 'fov': 30.0}
📨 recv:  {
  "type": "success",
  "message": "Camera view updated successfully"
}
✅ send: {'type': 'camera', 'command': 'setview', 'pos': [3.9945, 2.0, 0.2093], 'rot': [26.5651, -93.0, 0.0], 'fov': 30.0}
📨 recv:  {
  "type": "success",
  "message": "Camera view updated successfully"
}
✅ send: {'type': 'camera', 'command': 'setview', 'pos': [3.9877, 2.0, 0.3138], 'rot': [26.5651, -94.5, 0.0], 'fov': 30.0}
📨 recv:  {
  "type": "success",
  "message": "Camera view updated successfully"
}
✅ send: {'type': 'camera', 'command': 'setview', 'pos': [3.9781, 2.0, 0.4181], 'rot': [26.5651, -96.0, 0.0], 'fov': 30.0}
📨 recv:  {
  "type": "success",
  "message": "Camera view updated su